In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from matplotlib import pyplot as plt
import os

/hpcnfs/home/ieo5776/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
os.getcwd()

'/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/results'

In [3]:
# Set random seeds

torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [2]:
# Define data loading step

batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    MNIST('../data/mnist', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    MNIST('../data/mnist/', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 70597373.88it/s]


Extracting ../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 57301652.71it/s]

Extracting ../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25616310.08it/s]


Extracting ../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20266519.97it/s]


Extracting ../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw



In [10]:
print(len(train_loader.dataset))
a = next(iter(train_loader))
print(a[0].shape)


60000
torch.Size([256, 1, 28, 28])


In [10]:
# for y, x in train_loader:
# 	test_y = y
# 	test_x = x

test_y.shape

torch.Size([256, 1, 28, 28])

In [5]:
# Defining the device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# Defining the model

d = 20

class VAE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(784, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, d * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(d, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, 784),
            nn.Sigmoid(),
        )

    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, y):
        mu_logvar = self.encoder(y.view(-1, 784)).view(-1, 2, d)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterise(mu, logvar)
        return self.decoder(z), mu, logvar

model = VAE().to(device)

In [7]:
# Setting the optimiser

learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [8]:
# Reconstruction + KL divergence losses summed over all elements and batch

def loss_function(ỹ, y, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(
        ỹ, y.view(-1, 784), reduction='sum'
    )
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))

    return BCE + KLD

In [9]:
# Training and testing the VAE

epochs = 20
codes = dict(μ=list(), logσ2=list(), x=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        for y, _ in train_loader:
            y = y.to(device)
            # ===================forward=====================
            ỹ, mu, logvar = model(y)
            loss = loss_function(ỹ, y, mu, logvar)
            train_loss += loss.item()
            
            print(f"this is x_hat : {ỹ}\n-------------")
            print(f"this is x : {y}\n-------------")
            print(f"this is mu : {mu}\n-------------")
            print(f"this is logvar : {logvar}\n-------------")
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
    
    # # Testing
    
    # means, logvars, labels = list(), list(), list()
    # with torch.no_grad():
    #     model.eval()
    #     test_loss = 0
    #     for y, x in test_loader:
    #         y = y.to(device)
    #         # ===================forward=====================
    #         ỹ, mu, logvar = model(y)
    #         test_loss += loss_function(ỹ, y, mu, logvar).item()
    #         # =====================log=======================
    #         means.append(mu.detach())
    #         logvars.append(logvar.detach())
    #         labels.append(x.detach())
    # # ===================log========================
    # codes['μ'].append(torch.cat(means))
    # codes['logσ2'].append(torch.cat(logvars))
    # codes['x'].append(torch.cat(labels))
    # test_loss /= len(test_loader.dataset)
    # print(f'====> Test set loss: {test_loss:.4f}')
    # display_images(y, ỹ, 1, f'Epoch {epoch}')

this is x_hat : tensor([[0.5788, 0.4123, 0.2973,  ..., 0.3743, 0.5042, 0.5523],
        [0.5288, 0.4887, 0.4134,  ..., 0.4772, 0.4326, 0.5137],
        [0.5713, 0.5042, 0.3386,  ..., 0.4634, 0.5244, 0.5897],
        ...,
        [0.4896, 0.4205, 0.4327,  ..., 0.4666, 0.4717, 0.5052],
        [0.6277, 0.4940, 0.4740,  ..., 0.4565, 0.5402, 0.4278],
        [0.5421, 0.5051, 0.4856,  ..., 0.4427, 0.5078, 0.4596]],
       grad_fn=<SigmoidBackward0>)
-------------
this is x : tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 

KeyboardInterrupt: 